IMPORTS

In [ ]:
def load_libraries():
  import pandas as pd
  from sklearn.model_selection import train_test_split
  from tensorflow.keras.utils import normalize,to_categorical
  import tensorflow as tf
  from tensorflow.keras.models import Sequential
  from tensorflow.keras.layers import Dense, Dropout
  from sklearn.metrics import accuracy_score
  from tensorflow.keras.optimizers import Adam
  from tensorflow.keras import regularizers
  from sklearn.ensemble import RandomForestClassifier

DATA

In [ ]:
def setup_data(file_name):
  df = pd.read_csv(file_name).dropna()

  X = df.drop(columns="label")
  y = df['label']

  X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.2)

  X_train = normalize(X_train)
  X_test = normalize(X_test)
  y_train = to_categorical(y_train)
  y_test = to_categorical(y_test)

Build Network

In [ ]:
def build(num_of_layers,nodes_per_layer,activation_function,L2_lambda,Dropout_rate):
  tf.keras.backend.clear_session()

  List_of_models.append(Sequential())
  model = List_of_models[-1]

  random = tf.keras.initializers.RandomNormal(mean=0., stddev=1.)
  global num_input_nuerons, num_output_nuerons

  model.add(  Dense(num_input_nuerons)   )
  for i in range(num_of_layers):
    model.add(Dense(units=nodes_per_layer, activation=activation_function,kernel_initializer=random, kernel_regularizer = regularizers.l2(L2_lambda)))
    model.add(Dropout(Dropout_rate))

  model.add(Dense(units= num_output_nuerons , activation='softmax' ,kernel_initializer=random))

Callback

In [ ]:
class CustumCallback(tf.keras.callbacks.Callback):
  
  min_accuracy = .9
  counter = 0
  previous_accuracy = 0
  good_counter = 0

  def on_epoch_end(self, epoch, logs=None):
    if logs.get("accuracy") > self.min_accuracy:
      self.good_counter+=1
      if self.good_counter > 10:
        print(f"Accuracy consistently over {100*self.min_accuracy}%, training terminated.")
        self.model.stop_training = True
    else:
      self.good_counter = 0

    if epoch == 1: 
      pass
    elif self.previous_accuracy == logs.get("accuracy"):
      self.counter +=1
    else:
      self.counter = 0
    
    if self.counter == 10: 
      print(f"This network reached a maximum accuracy at {logs.get('accuracy')} ")
      self.model.stop_training = True

    self.previous_accuracy = logs.get("accuracy")

Training

In [ ]:
def train():
  model.compile(loss='categorical_crossentropy', optimizer="rmsprop", metrics=['accuracy'])
  model.fit(X_train, y_train, epochs=1000, batch_size=len(X_train), callbacks = [CustumCallback()])

  model.save(f'/content/drive/MyDrive/High School Ramaz/Meta-AI/Files/results/saved_model{Index_of_network}/')

Results of Nueral Netowrk

In [ ]:
def evaluateNN():
  y_train1d = [list(x).index(max(x)) for x in y_train]
  y_test1d = [list(x).index(max(x)) for x in y_test]

  y_hat1 = [list(x).index(max(x)) for x in model.predict(X_train)]
  training_accuracy = 100 * accuracy_score(y_train1d, y_hat1)
  print("NN Training Accuracy:", training_accuracy)
  cm = tf.math.confusion_matrix(y_train1d, y_hat1)
  print(cm)

  y_hat = [list(x).index(max(x)) for x in model.predict(X_test)]
  testing_accuracy = 100 * accuracy_score(y_test1d, y_hat)
  print("NN Testing Accuracy:",testing_accuracy) 
  cm1 = tf.math.confusion_matrix(y_test1d, y_hat)
  print(cm1)

  global Index_of_network
  ALL_NETWORKS.append({'Testing accuracy':testing_accuracy, 'Training accuracy':training_accuracy})
  Index_of_network += 1

Change the NN based on Data

In [ ]:
def analyze(num_of_layers,nodes_per_layer,activation_function,L2_lambda,Dropout_rate):
  test_accuracy = ALL_NETWORKS[-1]['Testing accuracy']
  train_accuracy = ALL_NETWORKS[-1]['Training accuracy']

  print(test_accuracy)

  if train_accuracy * .95 > test_accuracy :
    print("MOST LIKELY OVERFIT")
    num_of_layers += 1
    nodes_per_layer += 5
  elif test_accuracy < 99:#CustumCallback.min_accuracy:
    print("NOT TESTING HIGH ENOUGH ACCURACY")
    num_of_layers += 1
    nodes_per_layer += 5
  else: print("This is good")
  
  return num_of_layers,nodes_per_layer,activation_function,L2_lambda,Dropout_rate

Run

In [ ]:
Index_of_network = 0
ALL_NETWORKS = []
List_of_models = []
num_output_nuerons = len(pd.unique(y))
num_input_nuerons = X.shape[1]
num_of_layers = 10
nodes_per_layer = 100
activation_function = "sigmoid"
L2_lambda = 0
Dropout_rate = 0

def setup(file_name):
  load_libraries()
  setup_data(file_name)

def create_network(num_of_layers,nodes_per_layer,activation_function,L2_lambda,Dropout_rate):
  build(num_of_layers,nodes_per_layer,activation_function,L2_lambda,Dropout_rate)
  train()
  evaluateNN()

setup('/content/drive/MyDrive/High School Ramaz/Meta-AI/Files/train.csv')
for i in range(3):
  create_network(num_of_layers,nodes_per_layer,activation_function,L2_lambda,Dropout_rate)
  num_of_layers,nodes_per_layer,activation_function,L2_lambda,Dropout_rate = analyze(num_of_layers,nodes_per_layer,activation_function,L2_lambda,Dropout_rate)


Epoch 1/1000
1/1 [==============================] - 3s 3s/step - loss: 0.0012 - accuracy: 0.9999
Epoch 2/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1362 - accuracy: 0.9692
Epoch 3/1000
1/1 [==============================] - 2s 2s/step - loss: 0.5414 - accuracy: 0.9028
Epoch 4/1000
1/1 [==============================] - 2s 2s/step - loss: 0.1701 - accuracy: 0.9625
Epoch 5/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0821 - accuracy: 0.9829
Epoch 6/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0111 - accuracy: 0.9965
Epoch 7/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0025 - accuracy: 0.9995
Epoch 8/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0020 - accuracy: 0.9997
Epoch 9/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0018 - accuracy: 0.9998
Epoch 10/1000
1/1 [==============================] - 2s 2s/step - loss: 0.0016 - accuracy: 0.9998
Epoch 11/1000
1/1 [==========

In [ ]:
for i in range(len(ALL_NETWORKS)):
  print(i, ALL_NETWORKS[i])

0 {'Testing accuracy': 95.97619047619048, 'Training accuracy': 99.99107142857143}
1 {'Testing accuracy': 95.89285714285715, 'Training accuracy': 99.99404761904762}
2 {'Testing accuracy': 96.0, 'Training accuracy': 99.98511904761904}


Random Forest (for comparison)

In [ ]:
rf = RandomForestClassifier()
rf.fit(X_train,y_train)

tra = rf.predict(X_train)
tes = rf.predict(X_test)

y_hat2 = [list(x).index(max(x)) for x in tra]
print("RF Training Accuracy:",100 * accuracy_score(y_train1d, y_hat2))
cm2 = tf.math.confusion_matrix(y_train1d,y_hat2)
print(cm2)

y_hat3 = [list(x).index(max(x)) for x in tes]
print("RF Testing Accuracy:",100 * accuracy_score(y_test1d, y_hat3))
cm3 = tf.math.confusion_matrix(y_test1d,y_hat3)
print(cm3)